<a href="https://colab.research.google.com/github/sequoiazhao/submitcode/blob/master/2019Tensor_0298.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


# get data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

# 批次大小
batch_size = 100

n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

keep_prob = tf.placeholder(tf.float32)
lr = tf.Variable(0.001, dtype=tf.float32)

# create network

W1 = tf.Variable(tf.truncated_normal([784, 500], stddev=0.1))
b1 = tf.Variable(tf.zeros([500]) + 0.1)

L1 = tf.nn.tanh(tf.matmul(x, W1) + b1)

L1_drop = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.truncated_normal([500, 300], stddev=0.1))
b2 = tf.Variable(tf.zeros([300]) + 0.1)

L2 = tf.nn.tanh(tf.matmul(L1_drop, W2) + b2)
L2_drop = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.truncated_normal([300, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]) + 0.1)

prediction = tf.nn.softmax(tf.matmul(L2_drop, W3) + b3)


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

train_step = tf.train.AdamOptimizer(lr).minimize(loss)

init = tf.global_variables_initializer()

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(51):
        sess.run(tf.assign(lr, 0.001 * (0.95 ** epoch)))
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.0})

        learning_rate = sess.run(lr)
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
        print("Iter" + str(epoch) + ",Testing Accuracy=" + str(acc) + ", Learning Rate=" + str(learning_rate))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter0,Testing Accuracy=0.9483, Learning Rate=0.001
Iter1,Testing Accuracy=0.9625, Learning Rate=0.00095
Iter2,Testing Accuracy=0.9667, Learning Rate=0.0009025
Iter3,Testing Accuracy=0.972, Learning Rate=0.000857375
Iter4,Testing Accuracy=0.9715, Learning Rate=0.00081450626
Iter5,Testing Accuracy=0.9733, Learning Rate=0.0007737809
Iter6,Testing Accuracy=0.9739, Learning Rate=0.0007350919
Iter7,Testing Accuracy=0.9745, Learning Rate=0.0006983373
Iter8,Testing Accuracy=0.9765, Learning Rate=0.0006634204
Iter9,Testing Accuracy=0.9768, Learning Rate=0.0006302494
Iter10,Testing Accuracy=0.9774, Learning Rate=0.0005987369
Iter11,Testing Accuracy=0.9791, Learning Rate=0.0005688001
Iter12,Testing Accuracy=0.9791, Learning Rate=0.0005403601
Iter13,Testing Accuracy=0.9784, Learning Rate=0.0005133421
Iter14

可视化，构建tensorboard

In [0]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

# get data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)



# 批次大小
batch_size = 100

n_batch = mnist.train.num_examples // batch_size

#定义命名空间
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784],name="x-input")
    y = tf.placeholder(tf.float32, [None, 10],name='y-input')

#
with tf.name_scope("layer"):
    with tf.name_scope("weights"):
        W = tf.Variable(tf.zeros([784, 10]),name="W")
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros([10])+0.1,name="b")
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y - prediction))
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
# 梯度下降
# train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

init = tf.global_variables_initializer()

# 结果存放
with tf.name_scope("accuracy"):
    with tf.name_scope("prediction"):
        correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(prediction, 1))

    # 先将bool型转为float
    with tf.name_scope("acc"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

config = tf.ConfigProto(allow_soft_placement=True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
config.gpu_options.allow_growth = True

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(2):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
        #
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("iter " + str(epoch) + ",Testing Accuracy " + str(acc))

各种参数设置visible

In [0]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

# get data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

# 批次大小
batch_size = 100

n_batch = mnist.train.num_examples // batch_size


# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)


# 定义命名空间
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')

#
with tf.name_scope("layer"):
    with tf.name_scope("weights"):
        W = tf.Variable(tf.truncated_normal([784, 10]), name="W")
        variable_summaries(W)
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros([10]) + 0.1, name="b")
        variable_summaries(b)
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x, W) + b
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y - prediction))
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar("loss", loss)
# 梯度下降
# train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

init = tf.global_variables_initializer()

# 结果存放
with tf.name_scope("accuracy"):
    with tf.name_scope("prediction"):
        correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(prediction, 1))

    # 先将bool型转为float
    with tf.name_scope("acc"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

config = tf.ConfigProto(allow_soft_placement=True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
config.gpu_options.allow_growth = True

# 合并所有summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/', sess.graph)
    for epoch in range(51):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            summary, _ = sess.run([merged, train_step], feed_dict={x: batch_xs, y: batch_ys})

        writer.add_summary(summary, epoch)
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("iter " + str(epoch) + ",Testing Accuracy " + str(acc))


设置分类数据的可视化效果!
![替代文字](https://lh3.google.com/u/0/d/1HcJLpLLj6NqqylKSjwL33Vx_C6eLXg9y=w1920-h855-iv1)


In [0]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

# get data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

max_steps = 1001

image_num = 3000

sess = tf.Session()

embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name="embedding")


# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)


with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')

with tf.name_scope("input_reshpae"):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

with tf.name_scope("layer"):
    with tf.name_scope("weights"):
        W = tf.Variable(tf.truncated_normal([784, 10]), name="W")
        variable_summaries(W)
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros([10]) + 0.1, name="b")
        variable_summaries(b)
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x, W) + b
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)
# 二次代价函数
# loss = tf.reduce_mean(tf.square(y - prediction))
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar("loss", loss)
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

init = tf.global_variables_initializer()

# 结果存放
with tf.name_scope("accuracy"):
    with tf.name_scope("prediction"):
        correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(prediction, 1))

    # 先将bool型转为float
    with tf.name_scope("acc"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

# metadata
Dir = "/code/remotePy/com/Tensor/"
if tf.gfile.Exists(Dir + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(Dir + 'projector/projector/metadata.tsv')

with open(Dir + 'projector/projector/metadata.tsv', 'w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:], 1))
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')


config = tf.ConfigProto(allow_soft_placement=True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
config.gpu_options.allow_growth = True
# merged
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(Dir + 'projector/projector', sess.graph)
saver = tf.train.Saver()

config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = Dir + "projector/projector/metadata.tsv"
embed.sprite.image_path = Dir + "projector/data/mnist_10k_sprite.png"
embed.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(projector_writer, config)

sess.run(init)

for i in range(max_steps):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary, _ = sess.run([merged, train_step], feed_dict={x: batch_xs, y: batch_ys}, options=run_options,
                          run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
    projector_writer.add_summary(summary, i)

    if i % 100 == 0:
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Iter" + str(i) + ",Testing Accuracy=" + str(acc))

saver.save(sess, Dir + 'projector/projector/a_model.ckpt', global_step=max_steps)
projector_writer.close()
sess.close()



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.mat

FileNotFoundError: ignored